In [38]:
from random import seed
from random import randint
from bitstring import Bits

In [39]:
regDict = {
    '$zero' : '00000',
    '$at' : '00001',
    '$v0' : '00010',
    '$v1' : '00011',
    '$a0' : '00100',
    '$a1' : '00101',
    '$a2' : '00110',
    '$a3' : '00111',
    '$t0' : '01000',
    '$t1' : '01001',
    '$t2' : '01010',
    '$t3' : '01011',
    '$t4' : '01100',
    '$t5' : '01101',
    '$t6' : '01110',
    '$t7' : '01111',
    '$s0' : '10000',
    '$s1' : '10001',
    '$s2' : '10010',
    '$s3' : '10011',
    '$s4' : '10100',
    '$s5' : '10101',
    '$s6' : '10110',
    '$s7' : '10111',
    '$t8' : '11000',
    '$t9' : '11001',
    '$k0' : '11010',
    '$k1' : '11011',
    '$gp' : '11100',
    '$fp' : '11101',
    '$sp' : '11110',
    '$ra' : '11111'
}
opCode = {
    'lw' : '100011',
    'sw' : '101011',
    'add' : '000000',
    'sub' : '000000',
    'beq' : '000100',
    'j' : '000010',
    'nop': '000000',
    'addi' : '001000',
    'and' : '000000',
    'or' : '000000',
    'nor' : '000000',
    'andi' : '001100',
    'ori' : '001101',
    'sll' : '000000',
    'srl' : '000000',
    'bne' : '000101',
    'jr' : '000000',
    'jal' : '000011'
}
funct = {
    'add' : '100000',
    'sub' : '100010',
    'and' : '100100',
    'or' : '100101',
    'nor' : '100111',
    'sll' : '000000',
    'srl' : '000010',
    'jr' : '001000'
}

In [40]:
def InstructionCreater(instructionWithSpace):
    # print(instructionWithSpace)
    spaceSeperated = instructionWithSpace.split(' ')
    instruction = spaceSeperated[0]
    if instruction in ['lw', 'sw', 'beq', 'bne']: # for i formate        
        rt = spaceSeperated[1][:-1]
        if instruction in ['beq', 'bne']:
            rs  = spaceSeperated[2][:-1]
            cons = Bits(int=int(spaceSeperated[3])-1, length=16).bin # -1 is added for as already pc is added by 4 and to remove it
        else:
            bractetSplit = spaceSeperated[2].split('(')
            cons = Bits(int=int(bractetSplit[0])*4, length=16).bin
            rs = bractetSplit[1][:-1]        
        ins = hex(int('0b'+opCode[instruction]+regDict[rs]+regDict[rt]+cons,2))[2:] 
    elif instruction in ['add', 'sub', 'and', 'or', 'nor']: # for R format
        rd = spaceSeperated[1][:-1]
        rs = spaceSeperated[2][:-1]
        rt = spaceSeperated[3]
        ins = hex(int('0b'+opCode[instruction]+regDict[rs]+regDict[rt]+regDict[rd]+'00000'+funct[instruction], 2))[2:]
    elif instruction in ['j', 'jal']: # for J format
        addr = Bits(int=int(spaceSeperated[1])-1, length=26).bin     # -1 is added for as already pc is added by 4 and to remove it   
        ins = hex(int('0b'+opCode[instruction]+addr,2))[2:]        
    elif instruction in ['nop']:
        ins = hex(int('0b'+opCode['nop']+''.zfill(25),2))[2:]
    elif instruction in ['addi', 'andi', 'ori']:
        rs = spaceSeperated[2][:-1]
        rt = spaceSeperated[1][:-1]
        cons = Bits(int=int(spaceSeperated[3]), length=16).bin
        ins = hex(int('0b'+opCode[instruction]+regDict[rs]+regDict[rt]+cons,2))[2:]
    elif instruction in ['sll', 'srl']:
        rt = spaceSeperated[2][:-1]
        rd = spaceSeperated[1][:-1]
        sa = Bits(int=int(spaceSeperated[3]), length=5).bin
        ins = hex(int('0b'+opCode[instruction]+'00000'+regDict[rt]+regDict[rd]+sa+funct[instruction], 2))[2:]
    elif instruction in ['jr']:
        rs = spaceSeperated[1]
        ins = hex(int('0b'+opCode[instruction]+regDict[rs]+15*'0'+funct[instruction], 2))[2:]
    return ins
# InstructionCreater('sw $t5, 11($zero)')
# InstructionCreater('sub $s4, $t8, $t9')
# InstructionCreater('beq $zero, $zero, 7')
# InstructionCreater('j 15')

In [41]:
def writeDataMemory(depth=10, initwithzero=1, randomize=0, seedV=784):
    if initwithzero:
        st='00000000\n'*depth
    else:
        st = ''
        if randomize:
            seed(seedV)
            lis = [hex(randint(0, 40))[2:].zfill(8) for i in range(depth)]
            st = '\n'.join(lis)
    with open('MemFiles/DataMemory.mem', 'w+') as f:
        f.write(st)
        f.close()

In [44]:
def writeInstructionMemory():
    st = ''
    # ins 1 to 10
    for i in range(10):
        ins = InstructionCreater('lw $t'+str(i)+', '+str(i+0)+'($zero)')
        # ins = hex(int('0b'+opCode['lw']+regDict['$zero']+regDict['$t'+str(i)]+bin(i*4)[2:].zfill(16),2))[2:]    
        st =  st + ins + '\n'

    # chcking jr
    # # load s0 with 16
    # ins = InstructionCreater("addi $ra, $zero, 60")
    # st =  st + ins + '\n'

    # # jump to s0
    # ins = InstructionCreater("jr $ra")
    # st =  st + ins + '\n'


    ins = InstructionCreater("jal 15")
    st =  st + ins + '\n'


    ins = InstructionCreater("add $s0, $t0, $t1")
    st =  st + ins + '\n'

    ins = InstructionCreater("sub $s0, $t1, $t0")
    st =  st + ins + '\n'
    
    ins = InstructionCreater("and $s0, $t0, $t1")
    st = st + ins + '\n'

    ins = InstructionCreater("or $s0, $t0, $t1")
    st = st + ins + '\n'

    ins = InstructionCreater("nor $s0, $t0, $t1")
    st = st + ins + '\n'

    ins = InstructionCreater("addi $s0, $t0, 50")
    st =  st + ins + '\n'

    ins = InstructionCreater("andi $s0, $t0, 50")
    st =  st + ins + '\n'

    ins = InstructionCreater("ori $s0, $t0, 65")
    st =  st + ins + '\n'

    ins = InstructionCreater("sll $s0, $t0, 3")
    st =  st + ins + '\n'

    ins = InstructionCreater("srl $s0, $t0, 3")
    st =  st + ins + '\n'
   

    # ins = I

    for i in range(30-14):
        ins = InstructionCreater('nop')
        st =  st + ins + '\n'

    with open('MemFiles/InstructionMemory.mem', 'w+') as f:
        f.write(st)
        f.close()

In [45]:
writeInstructionMemory()
writeDataMemory(200,0,1, seedV=177)